# Analysis presented in plots
Here we create the plots that allow us to do the analysis of the results.

We import all the packages necesarry to plot the results from the models

In [1]:
%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import seaborn as sns
import astropy.units as u
from collections import Counter
import matplotlib.lines as mlines
from astropy.modeling import models
from astroquery.simbad import Simbad
from scipy.stats import binned_statistic
from astropy.table import Table, join, hstack

Additionally, we add some shortcuts

In [3]:
# User shortcuts
def Perc75(array):
    """75th Percentile"""
    return(np.percentile(array, 75))


def Perc25(array):
    """25th Percentile"""
    return(np.percentile(array, 25))

## Join all clean results

We read the clean results from the models we are going to use 

In [4]:
ModelS = Table.read('CIGALEOutputs/Cleanresults_SK.fits', format='fits')
ModelF = Table.read('CIGALEOutputs/Cleanresults_Fr.fits', format='fits')

ModelS37 = Table.read('CIGALEOutputs/Cleanresults_SK37.fits', format='fits')
ModelF37 = Table.read('CIGALEOutputs/Cleanresults_Fr37.fits', format='fits')

We also read the original database of galaxies to use the information for the classification

In [5]:
TV = Table.read('VCV_TAP_otype.txt', format='ascii')
TV.rename_column('main_id', 'id')  # Renamed to use the join with the same id

We select a list of parameters from the models to be plotted (Check also the Analysis_Tables file to check why we choose these parameters). We add the errors from these parameters also in the plots. Also, we define quantities useful for plots for each parameters.

In [6]:
ParamCheck = ['bayes.agn.i', 'bayes.agn.fracAGN', 'bayes.agn.disk_luminosity', 'bayes.agn.EBV',
              'bayes.attenuation.E_BVs', 'bayes.sfh.tau_main', 'bayes.sfh.sfr']
ParamUnit = ['[degrees]', 'Fraction', 'log([L$_{\odot}$])', 'mag',
             'mag', r'log([Myr])', r'log([M$_{\odot}$/yr])']
ParamLabel = ['Incl. Angle', r'$f_{\rm{AGN}}$', r'$\rm{L}_{\rm{AGN}}^{\rm{disk}}$',
              r'Polar-dust E(B-V)', 'E(B-V)', r'$\tau_{\rm{main}}$', 'SFR']
ParamRange = [[0, 90], [0.05, 0.95], [7, 13], [0, 1],
              [0, 0.4], [1.5, 4], [-3, 3]]
ParamCheckErr = [j+'_err' for j in ParamCheck]
ParamRangeErr = [[0, 40], [0.0, 0.4], [0, 1],
                 [0, 0.5], [0, 0.15], [0, 0.8], [0, 1]]
ParamUnitErr = ['[degrees]', 'Fraction', 'dex', 'mag', 'mag', 'dex', 'dex']
Particks = [[0,25,45,65,90],[0.25,0.5,0.75],[8,10,12],[0,0.4,0.8],
           [0,0.2,0.4],[2,3,4],[-2,0,2]]
ParticksErr = [[0,10,20,30],[0,0.1,0.2,0.3],[0.25,0.5,0.75],[0,0.2,0.4],[0,0.05,0.1],
           [0,0.2,0.4,0.6],[0,0.5,1]]

We join these clean tables with the catalog. We send all the tables to pandas format. Then, we concatenate the tables to have an unique table.

In [7]:
SPandas = join(TV, ModelS, keys='id').to_pandas()
FPandas = join(TV, ModelF, keys='id').to_pandas()
SPandas37 = join(TV, ModelS37, keys='id').to_pandas()
FPandas37 = join(TV, ModelF37, keys='id').to_pandas()
SFPandas = pd.concat([SPandas, FPandas, SPandas37, FPandas37], 
                     keys=['SKIRTOR', 'Fritz', 'S37', 'F37'], names=['Model', 'idCIG'])

We transform luminosities from W to solar luminosities. We reformat some parameters in log units, useful for the plots.

In [8]:
for kisd in ['bayes.agn.luminosity', 'bayes.agn.disk_luminosity', 'bayes.dust.luminosity']:
    SFPandas[kisd] = (SFPandas[kisd].values*u.W).to(u.solLum).value
    SFPandas[kisd+'_err'] = (SFPandas[kisd+'_err'].values *u.W).to(u.solLum).value

for asdf in ['bayes.agn.disk_luminosity', 'bayes.sfh.sfr', 'bayes.sfh.tau_main']:
    SFPandas[asdf+'_err'] = SFPandas[asdf+'_err']/(SFPandas[asdf] * log(10))
    SFPandas[asdf] = log10(SFPandas[asdf])

Finally, we create a definition that will help us to plot all the comparisons in the same way

In [9]:
def ParamFigure(PandaTable, HueMod, Method='scott', Order=None):
    """Figure used to present the results and comparisons for the selected parameters"""
    ii = 0
    LineSty = ['-', '--', ':']
    for iP, Parm in enumerate(ParamCheck):
        ii += 1
        subplot2grid((2, len(ParamCheck)), (0, ii-1))
        if len(HueMod) > 1:
            for KL in range(len(HueMod[1])):
                sns.kdeplot(data=PandaTable.loc[[HueMod[1][KL]]], x=Parm, hue=HueMod[0],
                            legend=False, hue_order=Order, clip=ParamRange[iP], bw_method=Method,
                            common_norm=False, ls=LineSty[KL])
            if ii > 1:
                plt.yticks([])
                plt.ylabel(None)
        else:
            sns.kdeplot(data=PandaTable, x=Parm, hue=HueMod[0], legend=False, hue_order=Order,
                        clip=ParamRange[iP], bw_method=Method, common_norm=False)
            if ii > 1:
                plt.yticks([])
                plt.ylabel(None)
        title(ParamLabel[ii-1], fontsize=12)
        xlabel(ParamUnit[ii-1])
        xlim(ParamRange[iP])
        xticks(Particks[iP])
    ii = 0
    for iP, Parm in enumerate(ParamCheckErr):
        ii += 1
        subplot2grid((2, len(ParamCheckErr)), (1, ii-1))
        if len(HueMod) > 1:
            for KL in range(len(HueMod[1])):
                sns.kdeplot(data=PandaTable.loc[[HueMod[1][KL]]], x=Parm, hue=HueMod[0],
                            legend=False, hue_order=Order, clip=ParamRangeErr[iP], 
                            bw_method=Method, common_norm=False, ls=LineSty[KL])
            if ii > 1:
                plt.yticks([])
                plt.ylabel(None)
        else:
            sns.kdeplot(data=PandaTable, x=Parm, hue=HueMod[0], legend=False, hue_order=Order,
                        clip=ParamRangeErr[iP], bw_method=Method, common_norm=False)
            if ii > 1:
                plt.yticks([])
                plt.ylabel(None)
        xlabel(ParamUnitErr[ii-1])
        xlim(ParamRangeErr[iP])
        xticks(ParticksErr[iP])
    subplots_adjust(top=0.9, bottom=0.1, left=0.1, right=0.9, hspace=0.4, wspace=0)

## Compare physical parameters between Fritz and SKIRTOR
##### FIGURE 3
We plot the density distribution of the parameters and the models used in this work, together with the density distributions of the errors derived by CIGALE. 

In [12]:
# with plt.style.context('seaborn-talk'):
with plt.style.context('seaborn-paper'):
    plt.rcParams.update({'font.size': 12})
    plt.rcParams.update({'axes.labelsize': 12})
    plt.rcParams.update({'legend.fontsize': 10})
    plt.rcParams.update({'ytick.labelsize': 10})
    plt.rcParams.update({'xtick.labelsize': 10})
    figure(figsize=(14.06, 3.77))
    ParamFigure(SFPandas, ['Model'])
    SKI = mlines.Line2D([], [], color='C0', label='SKIRTOR')
    Fri = mlines.Line2D([], [], color='C1', label='Fritz')
    S37 = mlines.Line2D([], [], color='C2', label='SK-30/70')
    F37 = mlines.Line2D([], [], color='C3', label='Fr-30/70')
    plt.legend(handles=[SKI, Fri, S37, F37], loc='upper right', fontsize=8)
    subplots_adjust(top=0.9, bottom=0.1, left=0.1,
                    right=0.9, hspace=0.6, wspace=0)
    savefig('Figures/F3.pdf', dpi=300, bbox_inches='tight')  # For Paper
#     savefig('Talk_Figures/F3.png',dpi=300,bbox_inches = 'tight') ## For Presentation

## Compare physical parameters with Seyfert type
##### FIGURE 4
We plot the density distribution of the parameters but this time separating Seyfert type classification. 

In [15]:
# with plt.style.context('seaborn-talk'):
with plt.style.context('seaborn-paper'):
    plt.rcParams.update({'font.size': 12})
    plt.rcParams.update({'axes.labelsize': 12})
    plt.rcParams.update({'legend.fontsize': 10})
    plt.rcParams.update({'ytick.labelsize': 10})
    plt.rcParams.update({'xtick.labelsize': 10})
    figure(figsize=(14.06, 3.77))
    Selc5_1 = np.logical_and(
        SFPandas['otype_txt'] == 'Sy1', SFPandas['Sp'] == 'S1')
    Selc5_2 = np.logical_and(
        SFPandas['otype_txt'] == 'Sy2', SFPandas['Sp'] == 'S2')
    Selc5 = np.logical_or(Selc5_1, Selc5_2)
    ParamFigure(SFPandas[Selc5], ['Sp', ['SKIRTOR', 'Fritz']])
    S1Cl = mlines.Line2D([], [], color='C0', label='S1_SK')
    S2Cl = mlines.Line2D([], [], color='C1', label='S2_SK')
    S1Sm = mlines.Line2D([], [], color='C0', label='S1_Fr', ls='--')
    S2Sm = mlines.Line2D([], [], color='C1', label='S2_Fr', ls='--')
    plt.legend(handles=[S1Cl, S2Cl, S1Sm, S2Sm], loc='upper right', fontsize=8)
    subplots_adjust(top=0.9, bottom=0.1, left=0.1,
                    right=0.9, hspace=0.6, wspace=0)
    savefig('Figures/F4.pdf',dpi=300,bbox_inches = 'tight')
#     savefig('Talk_Figures/F4.png', dpi=300, bbox_inches='tight')  # For Presentation

We include an alternative version to this plot for the 30/70 cases

In [17]:
# with plt.style.context('seaborn-talk'):
with plt.style.context('seaborn-paper'):
    plt.rcParams.update({'font.size': 12})
    plt.rcParams.update({'axes.labelsize': 12})
    plt.rcParams.update({'legend.fontsize': 10})
    plt.rcParams.update({'ytick.labelsize': 10})
    plt.rcParams.update({'xtick.labelsize': 10})
    figure(figsize=(14.06, 3.77))
    ParamFigure(SFPandas[Selc5], ['Sp', ['S37', 'F37']])
    S1Cl = mlines.Line2D([], [], color='C0', label='S1_SK_30/70')
    S2Cl = mlines.Line2D([], [], color='C1', label='S2_SK_30/70')
    S1Sm = mlines.Line2D([], [], color='C0', label='S1_Fr_30/70', ls='--')
    S2Sm = mlines.Line2D([], [], color='C1', label='S2_Fr_30/70', ls='--')
    plt.legend(handles=[S1Cl, S2Cl, S1Sm, S2Sm], loc='upper right', fontsize=8)
    subplots_adjust(top=0.9, bottom=0.1, left=0.1,
                    right=0.9, hspace=0.6, wspace=0)
    savefig('Figures/F4_Alt.pdf', dpi=300, bbox_inches='tight')
#     savefig('Talk_Figures/F4_Alt.png', dpi=300, bbox_inches = 'tight') ## For Presentation

## Compare VCV intermediate classifications
##### FIGURE 5
We plot the VCV-Osterbrock classification of Seyfert galaxies in the same way as before. (Only with SKIRTOR results) 

In [18]:
Selc7 = np.logical_or.reduce([SFPandas['Sp'] == 'S1.0', SFPandas['Sp'] == 'S1.2', 
                              SFPandas['Sp'] == 'S1.5', SFPandas['Sp'] == 'S1.8', 
                              SFPandas['Sp'] == 'S1.9'])

In [21]:
# with plt.style.context('seaborn-talk'):
with plt.style.context('seaborn-paper'):
    plt.rcParams.update({'font.size': 12})
    plt.rcParams.update({'axes.labelsize': 12})
    plt.rcParams.update({'legend.fontsize': 10})
    plt.rcParams.update({'ytick.labelsize': 10})
    plt.rcParams.update({'xtick.labelsize': 10})
    figure(figsize=(14.06, 3.77))
    ParamFigure(SFPandas[Selc7], ['Sp', ['SKIRTOR']], 
                Order=['S1.0', 'S1.2', 'S1.5', 'S1.8', 'S1.9'])
    O1 = mlines.Line2D([], [], color='C0', label='S1.0')
    O2 = mlines.Line2D([], [], color='C1', label='S1.2')
    O3 = mlines.Line2D([], [], color='C2', label='S1.5')
    O4 = mlines.Line2D([], [], color='C3', label='S1.8')
    O5 = mlines.Line2D([], [], color='C4', label='S1.9')
    plt.legend(handles=[O1, O2, O3, O4, O5], loc='upper right', fontsize=8)
    subplots_adjust(top=0.9, bottom=0.1, left=0.1,
                    right=0.9, hspace=0.6, wspace=0)
    savefig('Figures/F5.pdf', dpi=300, bbox_inches='tight')
#     savefig('Talk_Figures/F5.png', dpi=300, bbox_inches = 'tight') ## For Presentation

## Relation with the original detection wavelength
##### FIGURE 6
We plot the same parameters as before but only with the initial information of detection in one of the selected wavelengths. We create a string to just use the three types of X-ray, IR and Radio detections.

In [22]:
SFPandas['XIR'] = SFPandas["X_OT"].astype(str)+\
                  SFPandas["IR_OT"].astype(str)+SFPandas["Rad_OT"].astype(str)

In [23]:
Selc6_4 = SFPandas.loc['SKIRTOR']['XIR'] == '1.01.01.0'
Selc6_5 = SFPandas.loc['SKIRTOR']['XIR'] == '1.00.00.0'
Selc6_6 = SFPandas.loc['SKIRTOR']['XIR'] == '0.01.00.0'
Selc6_7 = SFPandas.loc['SKIRTOR']['XIR'] == '0.00.01.0'
Selc6 = np.logical_or.reduce([Selc6_5, Selc6_6, Selc6_7])
Selc6_ALT = np.logical_or.reduce([Selc6_4, Selc6_5, Selc6_6, Selc6_7])

# The total number of galaxies per selection
print(sum(Selc6_4), sum(Selc6_5), sum(Selc6_6), sum(Selc6_7))

410 896 1426 222


In [26]:
# with plt.style.context('seaborn-talk'):
with plt.style.context('seaborn-paper'):
    plt.rcParams.update({'font.size': 12})
    plt.rcParams.update({'axes.labelsize': 12})
    plt.rcParams.update({'legend.fontsize': 10})
    plt.rcParams.update({'ytick.labelsize': 10})
    plt.rcParams.update({'xtick.labelsize': 10})
    figure(figsize=(14.06, 3.77))
    ParamFigure(SFPandas.loc['SKIRTOR'][Selc6], ['XIR'])
    SKI = mlines.Line2D([], [], color='C0', label='Xray')
    Fri = mlines.Line2D([], [], color='C1', label='IR')
    F37 = mlines.Line2D([], [], color='C2', label='Radio')
    plt.legend(handles=[SKI, Fri, F37], loc='upper right', fontsize=8)
    subplots_adjust(top=0.9, bottom=0.1, left=0.1, right=0.9, hspace=0.6, wspace=0)
    savefig('Figures/F6.pdf', dpi=300, bbox_inches='tight')
#     savefig('Talk_Figures/F6.png',dpi=300,bbox_inches = 'tight') ## For Presentation

We include an alternative version of this plot including when all the wavelengths were observed. 

In [28]:
# with plt.style.context('seaborn-talk'):
with plt.style.context('seaborn-paper'):
    plt.rcParams.update({'font.size': 12})
    plt.rcParams.update({'axes.labelsize': 12})
    plt.rcParams.update({'legend.fontsize': 10})
    plt.rcParams.update({'ytick.labelsize': 10})
    plt.rcParams.update({'xtick.labelsize': 10})
    figure(figsize=(14.06, 3.77))
    ParamFigure(SFPandas.loc['SKIRTOR'][Selc6_ALT], ['XIR'])
    XIR = mlines.Line2D([], [], color='C0', label='All')
    Xr = mlines.Line2D([], [], color='C1', label='Xray')
    IR = mlines.Line2D([], [], color='C2', label='IR')
    Rad = mlines.Line2D([], [], color='C3', label='Radio')
    plt.legend(handles=[XIR, Xr, IR, Rad], loc='upper right', fontsize=8)
    subplots_adjust(top=0.9, bottom=0.1, left=0.1, right=0.9, hspace=0.6, wspace=0)
    savefig('Figures/F6_ALT.pdf',dpi=300,bbox_inches = 'tight')
#     savefig('Talk_Figures/F6_ALT.png', dpi=300, bbox_inches='tight')  # For Presentation

## Redshift evolution with AGN luminosities
##### FIGURE 7
We now center on how are the Seyfert distributed with redshift on this sample. We plot the redshift distribution of the final galaxies and in other panel compared with the luminosity coming from the AGN disk. We calculate the running median for this luminosity in terms of redshift.

In [33]:
RzBins = np.linspace(0.0, 1.0, 41)
ParmsY = log10((SPandas['bayes.agn.disk_luminosity'].values*u.W).to(u.solLum).value)
ParmsY2 = log10((SPandas['bayes.agn.luminosity'].values*u.W).to(u.solLum).value)

bin_means, bin_edges, binnumber = binned_statistic(SPandas['rvz_redshift'][SPandas['otype_txt'] == 'Sy1'],
                                                   ParmsY[SPandas['otype_txt'] == 'Sy1'], statistic='median', bins=RzBins)
bin_means2, bin_edges2, binnumber2 = binned_statistic(SPandas['rvz_redshift'][SPandas['otype_txt'] == 'Sy2'],
                                                      ParmsY[SPandas['otype_txt'] == 'Sy2'], statistic='median', bins=RzBins)
bin_means_75, bin_edges, binnumberx = binned_statistic(SPandas['rvz_redshift'][SPandas['otype_txt'] == 'Sy1'],
                                                       ParmsY[SPandas['otype_txt'] == 'Sy1'], statistic=Perc75, bins=RzBins)
bin_means2_75, bin_edges2, binnumberx = binned_statistic(SPandas['rvz_redshift'][SPandas['otype_txt'] == 'Sy2'],
                                                         ParmsY[SPandas['otype_txt'] == 'Sy2'], statistic=Perc75, bins=RzBins)
bin_means_25, bin_edges, binnumberx = binned_statistic(SPandas['rvz_redshift'][SPandas['otype_txt'] == 'Sy1'],
                                                       ParmsY[SPandas['otype_txt'] == 'Sy1'], statistic=Perc25, bins=RzBins)
bin_means2_25, bin_edges2, binnumberx = binned_statistic(SPandas['rvz_redshift'][SPandas['otype_txt'] == 'Sy2'],
                                                         ParmsY[SPandas['otype_txt'] == 'Sy2'], statistic=Perc25, bins=RzBins)

Lbin_means, Lbin_edges, binnumberx = binned_statistic(SPandas['rvz_redshift'][SPandas['otype_txt'] == 'Sy1'],
                                                      ParmsY2[SPandas['otype_txt'] == 'Sy1'], statistic='median', bins=RzBins)
Lbin_means2, Lbin_edges2, binnumberx = binned_statistic(SPandas['rvz_redshift'][SPandas['otype_txt'] == 'Sy2'],
                                                        ParmsY2[SPandas['otype_txt'] == 'Sy2'], statistic='median', bins=RzBins)
Lbin_means_75, bin_edges, binnumberx = binned_statistic(SPandas['rvz_redshift'][SPandas['otype_txt'] == 'Sy1'],
                                                        ParmsY2[SPandas['otype_txt'] == 'Sy1'], statistic=Perc75, bins=RzBins)
Lbin_means2_75, bin_edges2, binnumberx = binned_statistic(SPandas['rvz_redshift'][SPandas['otype_txt'] == 'Sy2'],
                                                          ParmsY2[SPandas['otype_txt'] == 'Sy2'], statistic=Perc75, bins=RzBins)
Lbin_means_25, bin_edges, binnumbex = binned_statistic(SPandas['rvz_redshift'][SPandas['otype_txt'] == 'Sy1'],
                                                       ParmsY2[SPandas['otype_txt'] == 'Sy1'], statistic=Perc25, bins=RzBins)
Lbin_means2_25, bin_edges2, binnumberx = binned_statistic(SPandas['rvz_redshift'][SPandas['otype_txt'] == 'Sy2'],
                                                          ParmsY2[SPandas['otype_txt'] == 'Sy2'], statistic=Perc25, bins=RzBins)

We also calculated the limit where we are going to use the running median. We limited to a minimum of 12 galaxies. After that we plot the galaxies as scatter plots. We do not plot galaxies with redshifts higher than 1.01, as they are only a few and do not add much to the plot.

In [34]:
GoodSy1 = histogram(SPandas['rvz_redshift'][SPandas['otype_txt'] == 'Sy1'], 
                    bins=RzBins)[0] >= 12
GoodSy2 = histogram(SPandas['rvz_redshift'][SPandas['otype_txt'] == 'Sy2'], 
                    bins=RzBins)[0] >= 12

ScatS1 = np.logical_and(SPandas['otype_txt'] == 'Sy1', SPandas['rvz_redshift'] > 0.8)
ScatS2 = np.logical_and(SPandas['otype_txt'] == 'Sy2', SPandas['rvz_redshift'] > 0.575)

In [30]:
## This number of galaxies are missing in the plot because they have a high redshift
sum(SPandas['rvz_redshift']>=1.01)

22

In [39]:
with plt.style.context('seaborn-talk'):
# with plt.style.context('seaborn-paper'):
    plt.rcParams.update({'font.size': 12})
    plt.rcParams.update({'axes.labelsize': 12})
    plt.rcParams.update({'legend.fontsize': 10})
    plt.rcParams.update({'ytick.labelsize': 10})
    plt.rcParams.update({'xtick.labelsize': 10})

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6.75, 9.44),
                                   sharex=True, gridspec_kw={'height_ratios': [1, 3]})
    ax2.scatter(SPandas['rvz_redshift'][ScatS1],
                ParmsY[ScatS1], alpha=0.2)
    ax2.scatter(SPandas['rvz_redshift'][ScatS2],
                ParmsY[ScatS2], alpha=0.2)

    ax2.plot(bin_edges[:-1][GoodSy1], bin_means[GoodSy1],
             c='C0', label=r'$\rm{L}_{\rm{AGN}}^{\rm{disk}}$ Sy1')
    ax2.plot(bin_edges2[:-1][GoodSy2], bin_means2[GoodSy2],
             c='C1', label=r'$\rm{L}_{\rm{AGN}}^{\rm{disk}}$ Sy2')
    ax2.fill_between(bin_edges[:-1][GoodSy1], bin_means_75[GoodSy1],
                     y2=bin_means_25[GoodSy1], color='C0', alpha=0.2)
    ax2.fill_between(bin_edges2[:-1][GoodSy2], bin_means2_75[GoodSy2],
                     y2=bin_means2_25[GoodSy2], color='C1', alpha=0.2)

    ax2.plot(Lbin_edges[:-1][GoodSy1], Lbin_means[GoodSy1],
             c='C0', ls=':', label=r'$\rm{L}_{\rm{AGN}}$ Sy1')
    ax2.plot(Lbin_edges2[:-1][GoodSy2], Lbin_means2[GoodSy2],
             c='C1', ls=':', label=r'$\rm{L}_{\rm{AGN}}$ Sy2')
    ax2.fill_between(Lbin_edges[:-1][GoodSy1], Lbin_means_75[GoodSy1],
                     y2=Lbin_means_25[GoodSy1], color='C0', alpha=0.2)
    ax2.fill_between(Lbin_edges2[:-1][GoodSy2], Lbin_means2_75[GoodSy2],
                     y2=Lbin_means2_25[GoodSy2], color='C1', alpha=0.2)

    ax2.set_xlabel(r'$\rm{Redshift}$')
    ax2.set_ylabel(r'$\log(\rm{Luminosity} [\rm{L}_{\odot}])$')

    ax1.hist(SPandas['rvz_redshift'][SPandas['otype_txt'] ==
                                     'Sy1'], histtype='step', bins=RzBins, label='Sy1', lw=2)
    ax1.hist(SPandas['rvz_redshift'][SPandas['otype_txt'] ==
                                     'Sy2'], histtype='step', bins=RzBins, label='Sy2', lw=2)
    # The mayority of galaxies that we are missing are due to bad fits
    ax1.set_ylabel('# Galaxies')
    ax1.set_xlim(-0.01, 1.01)
    ax2.set_ylim(8, 13.3)
    ax1.set_yscale('log')
    ax1.legend(loc='best')
    ax2.legend(loc='best')
    subplots_adjust(wspace=0,hspace=0)
#     savefig('Figures/F7.pdf',dpi=300,bbox_inches = 'tight')
    savefig('Talk_Figures/F7.png',dpi=300,bbox_inches = 'tight') ## For Presentation

##### Notebook info

In [40]:
%load_ext watermark
%watermark -a "Andres Ramos" -d -v -m
print('Specific Python packages')
%watermark -iv -w

Author: Andres Ramos

Python implementation: CPython
Python version       : 3.8.3
IPython version      : 7.16.1

Compiler    : GCC 7.3.0
OS          : Linux
Release     : 3.10.0-1160.el7.x86_64
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

Specific Python packages
pandas    : 1.2.0
matplotlib: 3.2.2
seaborn   : 0.11.1
astropy   : 4.2
numpy     : 1.19.5
json      : 2.0.9
autopep8  : 1.5.4
sys       : 3.8.3 (default, Jul  2 2020, 16:21:59) 
[GCC 7.3.0]
re        : 2.2.1
logging   : 0.5.1.2

Watermark: 2.1.0

